In [11]:
import json
import os
import re

input_dir = '../../gym/raw/articles_json/'


### JSON example
# {
#     "PAPER'S NUMBER OF TABLES": 5,   to be removed
#     "S1.T1": {                       the table_id
#         "caption": "Table 1. Existing",
#         "table": "<table id=\"S1.T1.1.1\" +</tr>\n</table>\n\n",
#         "footnotes": [
#             "1"
#         ],
#         "references": [
#             "A",
#         ],
#     },
# }

def read_json(file):
    with open(file, 'r') as f:
        data = json.load(f)
    return data


def clean_text(text):
    #cast to string
    text = str(text)
    text = text.strip()
    
    # Remove HTML tags and attributes
    text = re.sub(r'<[^>]*>', ' ', text)
    # De-accent
    #text = unidecode(text)
    # Remove special characters
    #text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

    # Remove multiple spaces and newlines
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n+', ' ', text)

    return text

In [12]:
# Creating a table with table_id, caption, footnotes, references

import pandas as pd

df = pd.DataFrame(
    columns=[
        'id',
        'file_id',
        'table_id',
        'table_body',
        'caption',
        'references'
    ]
)

for file in os.listdir(input_dir):
    data = read_json(input_dir + file)
    for table_id in data:
        file_id = file.split('.')[0] + '.' + file.split('.')[1]
        table = data[table_id]
        table_body = clean_text(table['table'])
        caption = clean_text(table['caption'])
        references = clean_text(' '.join(table['references']))
        df.loc[len(df)] = [len(df), file_id, table_id, table_body, caption, references]

df.head()

,id,file_id,table_id,table_body,caption,references
0,0,2302.12156,S5.T1,Dataset Methods M=10 M=20 M=40 IoT devices Lo...,TABLE I: Summary of per-client test accuracy u...,Table I shows that KD-PDFL enables the clients...
1,1,2007.12004,S6.T1,Metrics Real-time After 2 hours After 4 hours...,TABLE I: Performance Comparsion of RMSE For GC...,The results about RMSERMSE\mathrm{RMSE} are pr...
2,2,2409.08498v1,S1.T1,Attributes Definition from Previous Research ...,Table 1. Procedural Fairness Attributes. For e...,Table 1 presents these three attributes of pro...
3,3,2409.08498v1,S4.T2,No. Post &#160;&#160;&#160;&#160;&#160;1 Mexi...,Table 2. Rule-violating Posts that Survey Part...,To test how the availability and granularity o...
4,4,2409.08498v1,S4.T3,Primary category Subdivision category Child s...,Table 3. Rule Violation Classification Scheme ...,(1) Rule Violation Classification Scheme In ou...


In [13]:
df.to_csv('../../gym/raw/' + 'data.csv', index=True)